- SOLAMENTE COLUMNAS NUMERICAS, SI TENES CATEGORICAS O STRINGS DROPEALAS
- Si dunn_index no está instalada, puedes instalarla con pip install dunn-index.
- Medidas de distancia: Cambia entre 'euclidean', 'manhattan', etc., según el método de clustering.

In [ ]:
#metodo del codo para definicion de K
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def metodo_codo(datos, max_k=10):
    """
    Calcula la suma de cuadrados within (inercia) para varios valores de K.
    """
    inercia = []
    for k in range(1, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(datos)
        inercia.append(kmeans.inertia_)

    # Gráfico del método del codo
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, max_k + 1), inercia, marker='o')
    plt.xlabel('Número de Clusters (K)')
    plt.ylabel('Suma de Cuadrados Within (Inercia)')
    plt.title('Método del Codo para Definir K')
    plt.grid()
    plt.show()


In [ ]:
#aplicacion KMEANS
def aplicar_kmeans(datos, n_clusters):
    """
    Aplica el algoritmo de K-Means y muestra los clusters y sus valores.
    """
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(datos)

    # Mostrar los clusters
    print("\nClusters asignados a cada observación:")
    print(pd.Series(clusters).value_counts().sort_index())

    # Retornar los clusters y el modelo
    return clusters, kmeans


In [ ]:
#VISUALIZACION CLUSTERS EN GRAFICO
def visualizar_clusters_pca(datos, clusters):
    """
    Reduce los datos a 2D usando PCA y visualiza los clusters.
    """
    pca = PCA(n_components=2)
    datos_pca = pca.fit_transform(datos)

    plt.figure(figsize=(8, 6))
    plt.scatter(datos_pca[:, 0], datos_pca[:, 1], c=clusters, cmap='viridis', alpha=0.7)
    plt.xlabel('Componente Principal 1')
    plt.ylabel('Componente Principal 2')
    plt.title('Visualización de Clusters en 2D (PCA)')
    plt.colorbar(label='Cluster')
    plt.grid()
    plt.show()


In [ ]:
#APLICACION PAM CON DISTANCIA DE MANHATTAN
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import pairwise_distances

def aplicar_pam(datos, n_clusters):
    """
    Aplica PAM (K-Medoids) con distancia de Manhattan.
    """
    pam = KMedoids(n_clusters=n_clusters, metric='manhattan', random_state=42)
    clusters = pam.fit_predict(datos)

    print("\nClusters asignados a cada observación (PAM):")
    print(pd.Series(clusters).value_counts().sort_index())

    return clusters, pam


In [ ]:
#GRAFICO DE SILOHUETTE Y DUNN
from sklearn.metrics import silhouette_score
from dunn_index import dunn

def evaluar_clusters(datos, clusters, metric='euclidean'):
    """
    Calcula el puntaje de Silhouette y Dunn para los clusters.
    """
    silhouette = silhouette_score(datos, clusters, metric=metric)
    print(f"Puntaje de Silhouette: {silhouette:.4f}")

    dunn_score = dunn(pairwise_distances(datos, metric=metric), clusters)
    print(f"Índice de Dunn: {dunn_score:.4f}")

    return silhouette, dunn_score


In [ ]:
#COMPARACION DE METODOS : RAND Y JACCARD
from sklearn.metrics import adjusted_rand_score, jaccard_score

def comparar_metodos(clusters1, clusters2):
    """
    Compara dos métodos de clustering usando Rand y Jaccard.
    """
    rand_index = adjusted_rand_score(clusters1, clusters2)
    jaccard_index = jaccard_score(clusters1, clusters2, average='macro')

    print(f"Índice de Rand: {rand_index:.4f}")
    print(f"Índice de Jaccard: {jaccard_index:.4f}")

    return rand_index, jaccard_index


In [ ]:
#CLUSTERING JERARQUICO Y COEF DE COHPENETIC
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist

def clustering_jerarquico(datos, metodo):
    """
    Aplica clustering jerárquico y calcula el coeficiente cophenético.
    """
    Z = linkage(datos, method=metodo)
    coph_corr, _ = cophenet(Z, pdist(datos))

    print(f"Coeficiente cophenético ({metodo}): {coph_corr:.4f}")

    # Generar dendrograma
    plt.figure(figsize=(10, 7))
    dendrogram(Z)
    plt.title(f'Dendrograma (Método: {metodo})')
    plt.xlabel('Observaciones')
    plt.ylabel('Distancia')
    plt.grid()
    plt.show()

    return Z, coph_corr


In [ ]:
#adicional: INDICE DE HOPKINS PARA TENDENCIA
import numpy as np
from sklearn.neighbors import NearestNeighbors

def calcular_hopkins(X, n_samples=100):
    """
    Calcula el coeficiente de Hopkins para un dataset.
    X: Dataset (matriz de datos numéricos).
    n_samples: Número de muestras aleatorias a usar para el cálculo.
    Retorna:
    - Coeficiente de Hopkins.
    """
    # Verificar que el dataset sea suficientemente grande
    if X.shape[0] < n_samples:
        n_samples = X.shape[0] // 2

    # Seleccionar n_samples aleatorios de los datos
    np.random.seed(42)
    idx = np.random.choice(range(X.shape[0]), n_samples, replace=False)
    X_sample = X[idx]

    # Generar n_samples puntos aleatorios dentro del rango de los datos
    X_min, X_max = np.min(X, axis=0), np.max(X, axis=0)
    X_random = np.random.uniform(X_min, X_max, (n_samples, X.shape[1]))

    # Calcular las distancias al vecino más cercano en el dataset original
    nn = NearestNeighbors(n_neighbors=1).fit(X)
    distances_sample, _ = nn.kneighbors(X_sample, n_neighbors=1)
    distances_random, _ = nn.kneighbors(X_random, n_neighbors=1)

    # Calcular el coeficiente de Hopkins
    W = np.sum(distances_sample)
    U = np.sum(distances_random)
    H = W / (W + U)

    return H

# Ejemplo de uso
# df_numerico = df.select_dtypes(include=[np.number])  # Asegúrate de usar solo columnas numéricas
# hopkins = calcular_hopkins(df_numerico.values, n_samples=100)
# print(f"Coeficiente de Hopkins: {hopkins:.4f}")


In [ ]:
# Asegúrate de usar solo columnas numéricas
df_numerico = df.select_dtypes(include=[np.number])

# Calcular el coeficiente de Hopkins
hopkins = calcular_hopkins(df_numerico.values, n_samples=100)
print(f"Coeficiente de Hopkins: {hopkins:.4f}")

# Si el coeficiente es cercano a 1, procede con clustering
if hopkins > 0.75:
    print("Los datos tienen estructura de clustering. Continuando con el análisis...")
    metodo_codo(df_numerico, max_k=10)
else:
    print("Los datos no tienen una estructura clara de clustering. Revisa el dataset.")
